In [2]:
import os 
import time 
import json,math,sys,re
import pandas as pd 

In [3]:
qp_folder = "/Users/amycweng/Downloads/QP"
from dotenv import load_dotenv
env_path = '/Users/amycweng/DH/openai.env'
load_dotenv(dotenv_path=env_path)
OPENAI_API_KEY = os.getenv('SECRET_KEY')
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
inputfname = "preE_qp"
fname = f"{inputfname}_GPT"
data = pd.read_csv(f"{qp_folder}/{inputfname}.csv")
data = data.drop(columns=['0','1','2','4','5','Unnamed: 7'])
data = data.rename(columns = {'3': 'verse_id','6':'text','8':'verse_text','9':'my_label'})
data = data.to_dict(orient="records")

In [26]:
inputfname = "Vul_notPriorInCited"
# inputfname = "preE_Vul"
fname = f"{inputfname}_GPT"
data = pd.read_csv(f"{qp_folder}/{inputfname}.csv")
data = data.to_dict(orient="records")

In [27]:
for idx, item in enumerate(data): 
    item['index'] = idx 
with open(f"/Users/amycweng/DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'r') as f: 
    labels = json.load(f) 
data[196], len(data)

({'verse_id': "['Amos 7.11 (Geneva)']",
  'score': 0.9228106737,
  'text': 'Perditio tua ex te Israel, Ho. 13. that is, from vs, and of vs,',
  'verse_text': 'For thus Amos saith, Ieroboam shall die by the sworde, and Israel shalbe led away captiue out of their owne land.',
  'label': False,
  'index': 196},
 615)

In [6]:
EEPS_QP_assistant = client.beta.assistants.create(
  name="EEPS Quotations/Paraphrase Identifier",
  instructions="You are an English and Latin scholar working on Biblical language in 16th and 17th century texts from England. You can determine whether a given sentence is a quotation/paraphrase of another given Biblical verse in the SAME LANGUAGE. You give TRUE/FALSE outputs FOR EACH USER MESSAGE without any reasoning.",
  model="gpt-4o",
)

In [7]:
def add_QP(qp_pairs):
    qp = client.beta.threads.create(
        messages=[{"role": "user", 
                   "content": f"SENTENCE: '{qp_pair['text']}'; 'VERSE: {qp_pair['verse_text']}'"} for qp_pair in qp_pairs]
    )
    return qp.id

def run_QP(thread_id):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=EEPS_QP_assistant.id
    )
    return run.id  

def wait_for_run_completion(thread_id, run_id):
    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run_id
        )
        if run_status.status in ["completed", "failed", "cancelled"]:
            return run_status.status
        time.sleep(1)  

def get_thread_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    return [(msg.role, msg.content[0].text.value) for msg in messages.data]

In [22]:
from tqdm import tqdm 
start = 555 
end = len(data)
my_qp = data[start:end] 

for qp_pair in tqdm(my_qp): 
    thread_id = add_QP([qp_pair]) 
    run_id = run_QP(thread_id)  
    status = wait_for_run_completion(thread_id, run_id) 
    if status == "completed":
        responses = get_thread_messages(thread_id)
        for role, text in responses:
            if role == 'assistant': 
                labels[qp_pair['index']] = text 
    else:
        print(f"Run {qp_pair['index']} failed or was cancelled.")

with open(f"/Users/amycweng/DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f) 
# takes 30 mins for 250 entries; 11 mins for 100 

100%|██████████| 60/60 [07:38<00:00,  7.65s/it]


In [23]:
with open(f"/Users/amycweng/DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f)
len(labels),fname

(615, 'Vul_notPriorInCited_GPT')